# Initialization

In [1]:
import logging, os, sys, time

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

sys.path.insert(0, "/home/pleroy/DEV/posar")
import backprojection as tdbp
from posarmctools import getNotebookLogger
import posarutils.process.disp_PoSAR_img as disp
from posarutils.process.disp_PoSAR_img import OPTt
from posarutils.process.filtering import box_filter

sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia

# this is for the sceneReferencePoint.py module
sys.path.insert(0, "/home/pleroy/DATA/PoSAR-X/PIMA-1")

In [2]:
%matplotlib qt5
eps = np.finfo(float).eps

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
logger = getNotebookLogger(logging.INFO)

In [5]:
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"

In [6]:
lib = tdbp.Lib(libraryFilename)

# Load focusing configuration and data

In [7]:
focusingDir = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/FOCUSING"

ini = "2020_06_25_11_57_59_c2_gr_hann_3.438.ini"

focusing = tdbp.Focusing(os.path.join(focusingDir, ini), tag="Sa", alongTrack=1, pointsFile="ini", corr=True)

======= Conf
rec_dir /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/2020_06_25_11_57_59
======= Track
======= Load analytic signal...
load /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/OUT/2020_06_25_11_57_59/Sa_files_0_56_rampUp_hanning.npy
sa.shape (57000, 5000)
load /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/OUT/2020_06_25_11_57_59/avg_Sa_files_0_56_rampUp_hanning.npy
avg.shape (1, 5000)
existing analytic signal found and loaded
======= Load antenna positions...
load /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/OUT/2020_06_25_11_57_59/n_time_xyz_nav_a_corr.npy
xa.shape (57000,)
xa_mean = 1326868.86, ya_mean = 7218882.67, za_mean = 501.46
======= Load Digital Terrain Elevation Data...
get dataset
build meshgrids
build rectBivariateSpline
======= Scene
buildSceneGroundRange
======= Set focusingParameters
Nf 5000, Naz 57000, phi_a_deg 3.438, azRes 0.250
range from 0.00m to 1249.97m, resolution = 0.25m, oversampled = 0.025m, 


## Plot scene

In [ ]:
fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
ax.set_aspect('equal')
ax.plot(focusing.xa, focusing.ya)
ax.plot(x, y, '.k')
ax.grid()
focusing.addPtsEpsg(ax, ['c0', 'c1', 'c2', 'c3', 'c4'])
focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])
ax.plot(focusing.scene.targetX, focusing.scene.targetY, 'o', label="scene.target")
ax.plot(focusing.track.refX, focusing.track.refY, 'D', label="track.ref")
#ax.plot(focusing.scene.refX, focusing.scene.refY, 'D', label="scene.ref")
#ax.plot(focusing.scene.closestApproach[0], focusing.scene.closestApproach[1], 'o')

# Focalization

In [ ]:
t = time.time()

focusedImg  = np.zeros( (focusing.scene.nY, focusing.scene.nX), dtype=complex )
print( "focusedImg.shape = {}".format( focusedImg.shape ) )

lib.so.backProjectionOmpGroundRange_LETG(focusing.scene.X.reshape(focusing.scene.X.size), 
                                         focusing.scene.Y.reshape(focusing.scene.Y.size),
                                         focusing.sceneZ.reshape(focusing.sceneZ.size),
                                         focusing.r_over,
                                         focusing.sa.reshape(focusing.sa.size),
                                         focusing.xyz.reshape(focusing.xyz.size), 
                                         focusedImg.reshape(focusedImg.size),
                                         focusing.parameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

In [ ]:
myImg = tdbp.Img(focusedImg)

# Plot image

## imshow

In [ ]:
cmap = 'jet'

In [ ]:
left = focusing.conf.xMax
right = focusing.conf.xMin
bottom = focusing.conf.yMax
top = focusing.conf.yMin
extent = [left, right, bottom, top]
vmin = myImg.max_dB - 60 # 60 pour ground range, 40 pour slant range
vmax = myImg.max_dB
#vmin = myImg.med_dB - 15
#vmax = myImg.med_dB + 15

fig, ax0 = plt.subplots()
im0 = ax0.imshow(myImg.dB, cmap=cmap, vmin=vmin, vmax=vmax, extent=extent)
dia.addColorBar(im0, ax0)
ax0.set_aspect('equal')

## resample

In [ ]:
fig, ax0 = plt.subplots(1, 1, figsize=(10, 5))
im0 = ax0.imshow(myImg.dB, cmap='gray', vmin=0, vmax=40)
dia.addColorBar(im0, ax0)

## pcolormesh box_filter

In [ ]:
z = myImg.box_dB(3)

In [ ]:
fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
im = ax.pcolormesh(x, y, z, cmap='jet', shading='auto')
ax.set_aspect('equal')
#ax.plot(focusing.xa, focusing.ya)
plt.colorbar(im)
ax.grid()
#ax.plot(focusing.conf.ptsEpsg['corner'][0], focusing.conf.ptsEpsg['corner'][1], 'og', markeredgecolor = 'black')

In [ ]:
vmin = 0
vmax = 40

fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
im = ax.pcolormesh(x, y, z, cmap='jet', vmin=vmin, vmax=vmax, shading='auto')
ax.set_aspect('equal')
ax.plot(focusing.xa, focusing.ya)
plt.colorbar(im)
ax.grid()
#focusing.addPtsEpsg(ax, ['corner'])
#focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])

## Dynamic range

In [ ]:
imgDyn = myImg.dynamicRange(dynRange=70, db=1, size=3)

In [ ]:
fig, ax = plt.subplots(1,1)
im = ax.pcolormesh(x, y, z, cmap=cmap, vmin=-10, vmax=40, shading='auto')
dia.addColorBar(im, ax)
ax.grid()
focusing.addPtsEpsg(ax, ['c2'])
focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])

# Build GeoTIFF

In [ ]:
tiffName = os.path.join(focusing.out_dir, focusing.conf.name + ".tif")

In [ ]:
myTiff = tdbp.Tiff(tiffName, focusing, z, cmap='gray')

# Save focused image

In [ ]:
myImg.save(focusing)

In [ ]:
myImg.saveAlt(focusing)

In [ ]:
file = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/FOCUSING/2020_06_25_11_57_59/2020_06_25_11_57_59_sr_hann_3.438.data"
otherImg = np.fromfile(file, dtype='complex64').reshape(480, 700)